<a href="https://colab.research.google.com/github/hemankundu/indian-currency-recognizer/blob/master/Transfer_Learning_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
 
from tensorflow.keras.applications import VGG16

In [0]:
dim = 256

In [3]:
pre_trained_model = VGG16(input_shape = (dim, dim, 3), 
                                include_top = False, 
                                weights ='imagenet' )

for layer in pre_trained_model.layers:
  layer.trainable = False
  

W0924 14:04:01.632324 140304001013632 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


58900480/58889256 [==============================] - 5s 0us/step


In [4]:
pre_trained_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [6]:
last_layer = pre_trained_model.get_layer('block5_pool')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

('last layer output shape: ', (None, 8, 8, 512))


In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
drive_path = '/content/drive/My Drive/colab/currency/'
data_dir = drive_path + "dataset/"
classes = os.listdir(data_dir)

In [0]:
training_dir = drive_path + 'tmp/currency/training/'
testing_dir = drive_path + 'tmp/currency/testing/'
dev_dir = drive_path + 'tmp/currency/dev/'
dirs = [training_dir, dev_dir, testing_dir]

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(shear_range = 0.2, fill_mode='nearest' , brightness_range=(0.01, 0.08), rescale=1.0/255.0, 
                                      rotation_range=90, width_shift_range= 0.1, height_shift_range=0.1, zoom_range=0.1)
train_generator = train_datagen.flow_from_directory(training_dir, target_size=(dim, dim), batch_size=32)
dev_datagen = ImageDataGenerator(shear_range = 0.2, fill_mode='nearest' , brightness_range=(0.01, 0.08), rescale=1.0/255.0, 
                                      rotation_range=90, width_shift_range= 0.1, height_shift_range=0.1, zoom_range=0.1)
dev_generator = dev_datagen.flow_from_directory(dev_dir, target_size=(dim, dim), batch_size=16)
test_datagen = ImageDataGenerator(rescale=1.0/255.0, )
test_generator = test_datagen.flow_from_directory(dev_dir, target_size=(dim, dim), batch_size=16)


Found 2054 images belonging to 6 classes.
Found 512 images belonging to 6 classes.
Found 512 images belonging to 6 classes.


In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
# Flatten the output layer to 1 dimension
# x = layers.Conv2D(128, 3, padding='same', kernel_regularizer=regularizers.l2(0.1), activation='relu')(last_output)
# x = layers.MaxPool2D()(x)
# x = layers.Conv2D(128, 3, padding='same', kernel_regularizer=regularizers.l2(0.1), activation='relu')(x)
# x = layers.MaxPool2D()(x)
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense  (6, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = Adam(lr=0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])


In [0]:
model_name = "VGG16"

In [0]:
model.load_weights(drive_path + model_name + "_w.h5")

In [17]:
history = model.fit_generator(
            train_generator,
            validation_data = test_generator,
            epochs = 20)

Epoch 1/20
65/65 [==============================] - 144s 2s/step - loss: 1.7933 - acc: 0.1558 - val_loss: 1.7913 - val_acc: 0.1699
Epoch 2/20
65/65 [==============================] - 129s 2s/step - loss: 1.7915 - acc: 0.1782 - val_loss: 1.7911 - val_acc: 0.1992
Epoch 3/20
65/65 [==============================] - 128s 2s/step - loss: 1.7917 - acc: 0.1704 - val_loss: 1.7910 - val_acc: 0.1816
Epoch 4/20
65/65 [==============================] - 129s 2s/step - loss: 1.7917 - acc: 0.1680 - val_loss: 1.7910 - val_acc: 0.1855
Epoch 5/20
65/65 [==============================] - 128s 2s/step - loss: 1.7917 - acc: 0.1728 - val_loss: 1.7910 - val_acc: 0.1875
Epoch 6/20
65/65 [==============================] - 126s 2s/step - loss: 1.7916 - acc: 0.1641 - val_loss: 1.7910 - val_acc: 0.1934
Epoch 7/20
65/65 [==============================] - 128s 2s/step - loss: 1.7916 - acc: 0.1665 - val_loss: 1.7909 - val_acc: 0.1855
Epoch 8/20
65/65 [==============================] - 130s 2s/step - loss: 1.7913 - a

KeyboardInterrupt: ignored

In [0]:
with open(drive_path + model_name + ".json", "w") as f:
  f.write(model.to_json())

In [0]:
model.save_weights(drive_path + model_name + "_w.h5")

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

NameError: ignored

In [0]:
loaded_model = tf.keras.models.load_model(drive_path + model_name + '_auto.h5')
for layer_loaded, layer_built in zip(loaded_model.layers, built_model.layers):
   layer_built.set_weights(layer_loaded.get_weights())
#loaded_model.summary()
#model.load_weights(drive_path + 'model3.h5')

In [0]:
model.save(drive_path + model_name + '_full.h5')
model.save_weights(drive_path + model_name + '_w.h5')